In [12]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import seaborn as sns
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, precision_score, recall_score, confusion_matrix, f1_score
from sklearn.preprocessing import Binarizer
from sklearn.metrics import roc_curve

In [4]:
df =sns.load_dataset('titanic')
df_sp=df[['survived','pclass','age','sibsp','parch','fare']]
df_sp.dropna(inplace=True)
df_sp

C:\Users\yrh07\AppData\Local\Temp\ipykernel_5404\2426697525.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_sp.dropna(inplace=True)


,survived,pclass,age,sibsp,parch,fare
0,0,3,22.0,1,0,7.2500
1,1,1,38.0,1,0,71.2833
2,1,3,26.0,0,0,7.9250
3,1,1,35.0,1,0,53.1000
4,0,3,35.0,0,0,8.0500
...,...,...,...,...,...,...
885,0,3,39.0,0,5,29.1250
886,0,2,27.0,0,0,13.0000
887,1,1,19.0,0,0,30.0000
889,1,1,26.0,0,0,30.0000


In [5]:
X_train, X_test, y_train, y_test = train_test_split(df_sp.drop('survived',axis=1), df_sp['survived'], test_size=0.3, random_state=111)

In [6]:
#평가지표 함수
def get_clf_eval(y_test, pred):
    confusion = confusion_matrix(y_test, pred)
    accuracy = accuracy_score(y_test, pred)
    precision = precision_score(y_test, pred)
    recall = recall_score(y_test, pred)
    print('오차 행렬')
    print(confusion)
    print('정확도: {0:.4f}, 정밀도:{1:.4f}, 재현율:{2:.4f}'.format(accuracy, precision, recall))

In [7]:
lr_clf = LogisticRegression(solver='liblinear')

In [9]:
lr_clf.fit(X_train,y_train)
pred = lr_clf.predict(X_test)
get_clf_eval(y_test, pred)

오차 행렬
[[110  25]
 [ 43  37]]
정확도: 0.6837, 정밀도:0.5968, 재현율:0.4625


In [11]:
pred_proba = lr_clf.predict_proba(X_test)
pred_proba

array([[6.74067896e-01, 3.25932104e-01],
       [7.16997795e-01, 2.83002205e-01],
       [5.35951068e-01, 4.64048932e-01],
       [5.90120261e-01, 4.09879739e-01],
       [8.38459049e-01, 1.61540951e-01],
       [2.48198655e-01, 7.51801345e-01],
       [7.84887355e-01, 2.15112645e-01],
       [4.87331285e-01, 5.12668715e-01],
       [3.95401498e-01, 6.04598502e-01],
       [3.36824912e-01, 6.63175088e-01],
       [4.44821607e-01, 5.55178393e-01],
       [8.19371139e-01, 1.80628861e-01],
       [6.65718401e-01, 3.34281599e-01],
       [6.04909944e-01, 3.95090056e-01],
       [6.77010826e-01, 3.22989174e-01],
       [7.32751885e-01, 2.67248115e-01],
       [4.49864818e-01, 5.50135182e-01],
       [3.56780855e-01, 6.43219145e-01],
       [5.76496638e-01, 4.23503362e-01],
       [8.57827854e-01, 1.42172146e-01],
       [6.74980064e-01, 3.25019936e-01],
       [8.82538532e-01, 1.17461468e-01],
       [2.74450283e-01, 7.25549717e-01],
       [7.91675653e-01, 2.08324347e-01],
       [5.894446

In [13]:
from sklearn.preprocessing import Binarizer

tt_threshold = 0.5 # 임계값을 기준 0.5

pred_proba_1 =pred_proba[:,1].reshape(-1,1)
binarizer_tt=Binarizer(threshold=tt_threshold).fit(pred_proba_1)
tt_pred =binarizer_tt.transform(pred_proba_1)

get_clf_eval(y_test, tt_pred)

오차 행렬
[[110  25]
 [ 43  37]]
정확도: 0.6837, 정밀도:0.5968, 재현율:0.4625


In [14]:
thresholds= [0.3, 0.4, 0.5, 0.6, 0.7]

def get_eval_threshold(y_test, pred_proba_1, thresholds):
    for i in thresholds:
        binarizer=Binarizer(threshold=i).fit(pred_proba_1)
        pred =binarizer.transform(pred_proba_1)
        print('임계값의 따른 평가지표', i)
        get_clf_eval(y_test,pred)
        print('-------------------------------')

get_eval_threshold(y_test,pred_proba[:,1].reshape(-1,1),thresholds )

임계값의 따른 평가지표 0.3
오차 행렬
[[69 66]
 [11 69]]
정확도: 0.6419, 정밀도:0.5111, 재현율:0.8625
-------------------------------
임계값의 따른 평가지표 0.4
오차 행렬
[[94 41]
 [30 50]]
정확도: 0.6698, 정밀도:0.5495, 재현율:0.6250
-------------------------------
임계값의 따른 평가지표 0.5
오차 행렬
[[110  25]
 [ 43  37]]
정확도: 0.6837, 정밀도:0.5968, 재현율:0.4625
-------------------------------
임계값의 따른 평가지표 0.6
오차 행렬
[[123  12]
 [ 50  30]]
정확도: 0.7116, 정밀도:0.7143, 재현율:0.3750
-------------------------------
임계값의 따른 평가지표 0.7
오차 행렬
[[126   9]
 [ 62  18]]
정확도: 0.6698, 정밀도:0.6667, 재현율:0.2250
-------------------------------


## 임계값을 0.3부터 0.7까지 0.1씩 증가시키면

1. 정확도: 임계값이 증가할수록 정확도도 증가하다가 임계값 0.7부터 감소한다.
2. 정밀도: 임계값이 증가할수록 정밀도도 증가하다가 임계값 0.7부터 감소한다.
3. 재현율: 임계값이 증가할수록 감소한다.
4. f1: 임계값이 증가할수록 감소한다.